In [ ]:
#Importing the required modules
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from sklearn.neighbors import KernelDensity
from galpy.util.bovy_coords import *
from scipy.integrate import tplquad
import time
import sys
sys.path.append('..')

#Setting up the quasi-isothermal instance for galpy
from galpy.potential import MWPotential2014
from galpy.actionAngle import actionAngleAdiabatic
from galpy.df import quasiisothermaldf

aA = actionAngleAdiabatic(pot=MWPotential2014,c=True)
qdf = quasiisothermaldf(1./3.,0.2,0.1,1.,1.,pot=MWPotential2014,aA=aA,cutcounter=True)

In [ ]:
#Importing qdf load data
samples_qdf = np.load('../../mock_data/qdf/data/sampleV_at_(0.0,0.0,0.0)_epsilon=0.5_rect.npy')

print('#stars', len(samples_qdf))
print('min/max/mean (x)', min(samples_qdf[:,0]), max(samples_qdf[:,0]), np.mean(samples_qdf[:,0]))
print('min/max/mean (y)', min(samples_qdf[:,1]), max(samples_qdf[:,1]), np.mean(samples_qdf[:,1]))
print('min/max/mean (z)', min(samples_qdf[:,2]), max(samples_qdf[:,2]), np.mean(samples_qdf[:,2]))
print('min/max/mean (vx)', min(samples_qdf[:,3]), max(samples_qdf[:,3]), np.mean(samples_qdf[:,3]))
print('min/max/mean (vy)', min(samples_qdf[:,4]), max(samples_qdf[:,4]), np.mean(samples_qdf[:,4]))
print('min/max/mean (vz)', min(samples_qdf[:,5]), max(samples_qdf[:,5]), np.mean(samples_qdf[:,5]))

In [ ]:
#Defining the KDE function rather than importing to allow for quick and easy changes
def generate_KDE(inputs, ker, bw_multiplier=5):
    """
    NAME:
        generate_KDE
    
    PURPOSE:
        Given an NxM matrix for inputs and one of six avaliable ker strings, 
        outputs a function `input_DKE` that treats the density estimate as a 
        black box function that can be sampled.
    
    INPUT:
        inputs (ndarray) = An NxM matrix where N is the number of data 
                           points and M is the number of parameters.
        ker (string) = One of the 6 avaliable kernel types (gaussian, 
                       tophat, epanechnikov, exponential, linear, cosine).
        selection = a selection function that takes parallax to Sun and returns
                    fraction of stars that are left after selection;
                    takes array; takes parallax in physical units
    
    OUTPUT:
        input_KDE (function) = A blackbox function for the density estimate
                               used for sampling data.
                               
    HISTORY:
        2018-07-15 - Updated - Ayush Pandhi
    """
    #Scaling velocities with z-score
    inputs_std = np.nanstd(inputs, axis=0)
    i1, i2, i3, i4, i5, i6 = np.mean(inputs, axis=0)
    inputs_mean = np.hstack((i1, i2, i3, i4, i5, i6))
    inputs = (inputs - inputs_mean)/inputs_std
    
    #Optimizing bandwidth in terms of Scott's Multivariate Rule of Thumb
    N = inputs.shape[0]
    bw = bw_multiplier * np.nanstd(inputs) * N ** (-1/10.)
    
    #Fit data points to selected kernel and bandwidth
    kde = KernelDensity(kernel=ker, bandwidth=bw).fit(inputs)  
    
    def input_KDE(samples):
        """
        NAME:
            input_KDE
    
        PURPOSE:
            Given a QxM matrix for samples, evaluates the blackbox density
            estimate function at those points to output a 1xQ array of 
            density values.
    
        INPUT:
            samples (ndarray) = A QxM matrix where Q is the number of points 
                                at which the kde is being evaluated and M is 
                                the number of parameters.
                                
        OUTPUT:
            dens (ndarray) = A 1xQ array of density values for Q data points.
                               
        HISTORY:
            2018-07-15 - Updated - Ayush Pandhi
        """      
        #Converting cylindrical samples to cartesian
        x, y, z = cyl_to_rect(samples[:,0], samples[:,1], samples[:,2])
        vx, vy, vz = cyl_to_rect_vec(samples[:,3], samples[:,4], samples[:,5], samples[:, 1])
        samples = np.stack((x, y, z, vx, vy, vz), axis=1)
        
        #Scaling samples with standard deviation
        samples = (samples - inputs_mean)/inputs_std
        
        #Get the log density for selected samples and apply exponential to get normal probabilities
        log_dens = kde.score_samples(samples)
        dens = np.exp(log_dens)
        
        #Return a 1xQ array of normal probabilities for the selected sample
        return dens
    
    #Return a black box function for sampling
    return input_KDE

In [ ]:
#Generate kde for 6D qdf inputs
kde_qdf_epanechnikov = generate_KDE(samples_qdf, 'epanechnikov')

In [ ]:
#Input values for R
R_input = np.linspace(7, 9, 100)

# -----------------
# Integrate over R
# -----------------

def kde_qdf_cyl_R(vT, vz, vR, R, phi, z):
    evaluation = kde_qdf_epanechnikov(np.array([[R, phi, z, vR, vT, vz]]))
    return evaluation

def integrate_over_R(kde_qdf_epanechnikov, R): #bounds of vT = [0, 300], bounds of vz = [-100, 100], bounds of vR = [-200, 200]
    print ("Evaluating at R =", R)
    return tplquad(kde_qdf_cyl_R, 0., 300., -100., 100., -200., 200., args=([R, 3.14, 0.]), epsabs=0.1)  # args=(R, phi, z)

R_output = np.ones([100])

print("With phi, z = 3.14, 0., integrating over vT from [0, 300], vz from [-100, 100] and vR from [-200, 200]")
print("KDE Evaluations along R from [7, 9] kpc over 100 subintervals.")
print()

counter = 0
start_overall = time.time()

for r in np.nditer(R_input):
    counter += 1
    print("Evaluation:", counter)
    start = time.time()
    R_output[counter - 1], error = integrate_over_R(kde_qdf_epanechnikov, r)
    print("Value:", R_output[counter - 1])
    end = time.time()
    print("Time to integrate: "'{}'"s".format(round(end - start, 2)))
    print("Time elapsed: "'{}'" min".format(round((end - start_overall)/60, 2)))
    print()

#Input values for z
z_input = np.linspace(-1, 1, 100)

# -----------------
# Integrate over z
# -----------------

def kde_qdf_cyl_z(vT, vz, vR, R, phi, z):
    evaluation = kde_qdf_epanechnikov(np.array([[R, phi, z, vR, vT, vz]]))
    return evaluation

def integrate_over_z(kde_qdf_epanechnikov, z): #bounds of vT = [0, 300], bounds of vz = [-100, 100], bounds of vR = [-200, 200]
    print ("Evaluating at z =", z)
    return tplquad(kde_qdf_cyl_z, 0., 300., -100., 100., -200., 200., args=([8.3, 3.14, z]), epsabs=0.1)  # args=(R, phi, z)

z_output = np.ones([100])

print("With phi, R = 3.14, 8.3, integrating over vT from [0, 300], vz from [-100, 100] and vR from [-200, 200]")
print("KDE Evaluations along z from [-1, 1] kpc over 100 subintervals.")
print()

counter = 0
start_overall = time.time()

for zstep in np.nditer(z_input):
    counter += 1
    print("Evaluation:", counter)
    start = time.time()
    z_output[counter - 1], error = integrate_over_z(kde_qdf_epanechnikov, zstep)
    print("Value:", z_output[counter - 1])
    end = time.time()
    print("Time to integrate: "'{}'"s".format(round(end - start, 2)))
    print("Time elapsed: "'{}'" min".format(round((end - start_overall)/60, 2)))
    print()

In [ ]:
rs2 = np.linspace(7.5*u.kpc, 8.5*u.kpc, 100)
rs_eval2 = np.array([qdf.density(r, 0.) for r in rs2])

zs2 = np.linspace(-0.5*u.kpc, 0.5*u.kpc, 100)
zs_eval2 = np.array([qdf.density(1., z) for z in zs2])

#Plotting evaluated densities over R
plt.figure(figsize=(10,8))
plt.plot(R_input, R_output/np.sum(R_output)/(R_input[1] - R_input[0]), label='kde')
plt.plot(rs2, rs_eval2/np.sum(rs_eval2)/(rs2[1] - rs2[0]), label = 'qdf')
plt.title('Integrated Velocities Over R')
plt.xlabel('R (kpc)')
plt.ylabel('Normalized Density')
plt.show()

#Plotting evaluated densities over z
plt.figure(figsize=(10,8))
plt.plot(z_input, z_output/np.sum(z_output)/(z_input[1] - z_input[0]), label='kde')
plt.plot(zs2, zs_eval2/np.sum(zs_eval2)/(zs2[1] - zs2[0]), label='qdf')
plt.title('Integrated Velocities Over z')
plt.xlabel('z (kpc)')
plt.ylabel('Normalized Density')
plt.show()